<a href="https://colab.research.google.com/github/mohamedehab00/A-Hybrid-Arabic-Text-Summarization-Approach-based-on-Transformers/blob/main/TextClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
st = ISRIStemmer()
nltk.download('stopwords')
stop=stopwords.words('arabic')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data Preprocessing


In [2]:

def clean(text):
  #remove all English chars
  text = re.sub(r'\s*[A-Za-z]\s*', ' ' , text)
  #remove hashtags
  text = re.sub("#", " ", text)
  #remove all numbers
  text = re.sub(r'\[0-9]*\]',' ',text)
  #remove duplicated chars
  text = re.sub(r'(.)\1+', r'\1', text)
  #remove :) or :(
  text = text.replace(':)', "")
  text = text.replace(':(', "")
  #remove multiple exclamation
  text = re.sub(r"(\!)\1+", ' ', text)
  #remove multiple question marks
  text = re.sub(r"(\?)\1+", ' ', text)
  #remove multistop
  text = re.sub(r"(\.)\1+", ' ', text)
  #remove additional spaces
  text = re.sub(r"[\s]+", " ", text)
  text = re.sub(r"[\n]+", " ", text)

  return text

def remStopWords(Text):
  return " ".join(word for word in Text.split() if word not in stop)

def stemWords(Text):
  return " ".join(st.stem(word) for word in Text.split())

In [3]:
#read data
dataFrame = pd.read_csv('/content/Sampled_File_with_SMOTE.csv')

dataFrame['text']=dataFrame['text'].apply(lambda Text: remStopWords(str(Text)))
dataFrame['text'] = dataFrame['text'].apply(lambda Text : clean(Text))
dataFrame['text']=dataFrame['text'].apply(lambda Text: stemWords(Text))

dataFrame.drop_duplicates(subset ="text",keep = 'first', inplace = True)

train_x, valid_x, train_y, valid_y = train_test_split(dataFrame['text'], dataFrame['class'],test_size=0.2,random_state=11)

before_encode_valid_y = dataFrame['class'].unique()

encoder = preprocessing.LabelEncoder()

train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)

tfidf_vect.fit(dataFrame['text'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

# Model Training

In [4]:
def train_model(classifier, feature_vector_train, label,feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(predictions, valid_y),predictions

In [5]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf_svm = GridSearchCV(SVC(), params_grid, cv=5)

In [6]:
accuracy, predictions = train_model(clf_svm,xtrain_tfidf, train_y, xvalid_tfidf)

print("accuracy: ",accuracy*100)

accuracy:  97.56309834638816


# Model To Pickle

In [9]:
import os

# Create the directory if it doesn't exist
directory = '/content/Pickles/'
os.makedirs(directory, exist_ok=True)

# Now you can proceed with opening the pickle files
pklClassifierFile = open('/content/Pickles/TextClassifier.pkl', 'wb')
pklEncodingFile = open('/content/Pickles/LabelEncoder.pkl', 'wb')
pklVectorizerFile = open('/content/Pickles/TextVectorizer.pkl', 'wb')

# source, destination
pickle.dump(clf_svm, pklClassifierFile)
pklClassifierFile.close()
# source, destination
pickle.dump(encoder, pklEncodingFile)
pklEncodingFile.close()
# source, destination
pickle.dump(tfidf_vect, pklVectorizerFile)
pklVectorizerFile.close()



# Test Of Generated Pickles

In [10]:
clf_svm = pickle.load(open('/content/Pickles/TextClassifier.pkl', 'rb'))

encoder = pickle.load(open('/content/Pickles/LabelEncoder.pkl', 'rb'))

tfidf_vect = pickle.load(open('/content/Pickles/TextVectorizer.pkl', 'rb'))


In [11]:
textInput = "كرة القدم من أكثر الالعاب الرياضية انتشارا في العالم وأقدمها[بحاجة لمصدر]. يعتبر كأس العالم لكرة القدم أكبر محفل دولي في مجال هذه اللعبة الرياضية و يطمح كل بلد في العالم في الحصول عليه.كرة القدم رياضة يتبارى فيها فريقان في ملعب ذو أرضية مزروعة على جانبيه مرميان ، و الفكرة في كرة القدم هي محاولة إيداع الكرة في مرمى الخصم ؛ و ذلك باستخدام الأرجل و الرأس غالبا ، حيث لا يسمح باستخدام اليدين إلا لحارس المرمى . يتكون كل فريق من 11 لاعبا ، يتواجد أثناء المباريات الرسمية حكم ساحة و مساعدين اثنين له على جوانب الملعب كرة القدم تلعب على مستوى احترافي في كل أنحاء العالم ، و الآلاف من عشاقها يذهبون إلى الملاعب لتشجيع فرقهم المفضلة ، بينما الملايين من الناس يشاهدون هذه الرياضة على التلفاز . عدد كبير جدا من الناس أيضا يمارسون رياضة كرة القدم بمستويات أقل احترافا . يذكر أن المباراة النهائية لكأس العالم لكرة القدم 2002 شاهدها أكثر من مليار و ثمان مئة مليون شخص حول العالم وهو ما يقارب نسبة 28% من سكان الكرة الأرضية يحكم كرة القدم الاتحاد الدولي لكرة القدم -الفيفا- و الذي يقوم بتنظيم هذه اللعبة وقوانينها حول العالم . وفقا للإحصائيات التي أعلنتها الفيفا FIFA في ربيع عام 2001 ، فإن أكثر من 240 مليون شخص يلعبون كرة القدم بانتظام في أكثر من 200 بلدة في كل أنحاء العالم[1].تقام العديد من بطولات كرة القدم أهمها على الاطلاق بطولة كأس العالم والتي تقام كل أربع سنوات ثم كأس الأمم الأوربية ودوري أبطال أوروبا وهناك أيضا بطولة الأمم الافريقية وبطولة الأمم الأسيوية وكوبا أمريكا وكأس العالم للشباب و غيرها."

In [12]:
def pipeline(Text):
  #preprocessing step
  Text = clean(Text)
  Text = "".join([char for char in Text if char not in string.ascii_letters]).strip()
  Text = remStopWords(str(Text))
  Text = stemWords(Text)
  #vectorize the text
  Text_Vector = tfidf_vect.transform([Text])
  predictions = clf_svm.predict(Text_Vector)
  return encoder.inverse_transform(predictions)[0]

In [13]:
print(pipeline(textInput))
print("DONE!!!")

رياضة
DONE!!!
